In [1]:
# 自动计算cell的计算时间
%load_ext autotime

time: 530 µs (started: 2021-08-03 08:17:26 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb

git remote -v

git commit -m '更新 ch11 #1 change Aug 01, 2021'

git push origin master

origin	git@github.com:ustchope/dlwpn.git (fetch)
origin	git@github.com:ustchope/dlwpn.git (push)
[master 64ccb0b] 更新 ch11 #1 change Aug 01, 2021
 2 files changed, 375 insertions(+), 239 deletions(-)
 create mode 100644 ch11.ipynb


To github.com:ustchope/dlwpn.git
   14d8f38..64ccb0b  master -> master


time: 5.76 s (started: 2021-08-01 22:16:47 +08:00)


In [2]:
#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[1] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

time: 5.77 s (started: 2021-08-03 08:17:26 +08:00)


In [3]:
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 23 ms (started: 2021-08-03 08:17:32 +08:00)


# 文本深度学习

> **本章包含**
> * 为机器学习应用程序预处理文本数据
> * 用于文本处理的词袋方法和序列建模方法
> * 变压器架构
> * 序列到序列学习

## 自然语言处理：鸟瞰图

在计算机科学中，我们将人类语言（如英语或普通话）称为“自然”语言，以区别于专为机器设计的语言，如汇编、LISP 或 XML。每一种机器语言都是：它的起点是人类工程师设计的写下一组正式规则来描述你可以用这种语言做出什么陈述，以及它们的含义。规则首先出现，人们只有在规则集完成后才开始使用该语言。对于人类语言，情况正好相反：使用在先，规则在后。自然语言是由进化过程塑造的，就像生物有机体一样——这就是它“自然”的原因。它的“规则”，就像英语的语法一样，事后正式化，经常被用户忽视或破坏。因此，虽然机器可读语言是高度结构化和严谨的，使用精确的句法规则从固定的词汇表中将精确定义的概念编织在一起，但自然语言是混乱的——模棱两可、混乱、庞大且不断变化。

创建能够理解自然语言的算法是一件大事：语言，尤其是文本，是我们大部分交流和大部分文化生产的基础。互联网主要是文本。语言是我们存储几乎所有知识的方式。我们的思想很大程度上建立在语言上。然而，理解自然语言的能力长期以来一直是机器所不具备的。有些人曾经天真地认为你可以简单地写下“英语规则集”，就像可以写下LISP的规则集一样。因此，构建自然语言处理 (NLP) 系统的早期尝试是通过“应用语言学”的视角进行的。工程师和语言学家会手工制作复杂的规则集来执行基本的机器翻译或创建简单的聊天机器人——比如 1960 年代著名的 ELIZA 程序，它使用模式匹配来维持非常基本的对话。但是语言是一种叛逆的东西，它不容易被形式化。经过几十年的努力，这些系统的能力仍然令人失望。

手工制定的规则一直是 1990 年代的主导方法。但从 1980 年代后期开始，更快的计算机和更高的数据可用性开始使更好的替代方案变得可行。当您发现自己构建的系统包含大量临时规则时，作为一名聪明的工程师，您可能会开始问：我可以使用数据语料库来自动化查找这些规则的过程吗？我可以在某种规则空间中搜索规则，而不必自己想出来吗？就这样，你已经毕业从事机器学习。所以在 1980 年代后期，我们开始看到机器学习方法用于自然语言处理。最早的那些是基于决策树的——其意图实际上是使以前系统的那种 if/then/else 规则的开发自动化。然后统计方法开始加速，从逻辑回归开始。随着时间的推移，学习的参数模型完全接管了，语言学开始被视为更多的障碍而不是有用的工具。早期的语音识别研究员 Frederick Jelinek 在 1990 年代开玩笑说：“每次我解雇一名语言学家，语音识别器的性能都会上升”。

这就是现代 NLP 的意义所在：使用机器学习和大型数据集赋予计算机能力——不是语言，这是一个更崇高的目标——而是摄取一段理解语言作为输入并返回一些有用的东西，比如预测：
* “这篇课文的主题是什么？” （文字分类）
* “这段文字是否包含辱骂？” （内容过滤）
* “这段文字听起来是正面的还是负面的？” （情绪分析）
* “这不完整的句子中的下一个单词应该是什么？” （语言建模）
* “这个用德语怎么说？” （翻译）
* “你如何用一段话概括这篇文章？” （总结）
* 等等。

当然，在本章中请记住，您将训练的文本处理模型并不具备人类对语言的理解； 相反，他们只是在输入数据中寻找统计规律——结果证明这足以在许多简单的任务上表现良好。 就像计算机视觉是应用于像素的模式识别一样，NLP 是应用于单词、句子和段落的模式识别。

NLP 的工具集——决策树、逻辑回归——从 1990 年代到 2010 年代初期才经历了缓慢的演变。 大多数研究重点是特征工程。 当我在 2013 年在 Kaggle 上赢得我的第一场 NLP 比赛时，我的模型，你猜对了，基于决策树和逻辑回归。 然而，在 2014-2015 年左右，事情终于开始发生变化。 多位研究人员开始研究循环神经网络的语言理解能力，特别是 LSTM——一种 1990 年代后期的序列处理算法，直到那时才受到关注

2015 年初，Keras 推出了第一个开源、易于使用的 LSTM 实现，这恰好是对循环神经网络重新产生兴趣的浪潮的开始——直到那时，只有“研究代码” 不能轻易重复使用。 然后从 2015 年到 2017 年，循环神经网络主导了蓬勃发展的 NLP 场景。 特别是双向 LSTM 模型在许多重要任务上设定了最新技术，从摘要到问答再到机器翻译。

最后，在 2017-2018 年左右，出现了一种取代 RNN 的新架构：Transformer，您将在本章的后半部分了解它。 Transformers 在短时间内在整个领域取得了长足的进步，如今大多数 NLP 系统都是基于它们的。

让我们深入了解细节。 本章将带您从最基础的知识到使用 Transformer 进行机器翻译。

## 准备文本数据

作为可微函数的深度学习模型只能处理数字张量：它们不能将原始文本作为输入。 text 是将文本转换为数字向量化张量的过程。 文本矢量化过程有多种形状和形式，但它们都遵循相同的模板：
* 首先，您将文本标准化以使其更易于处理，例如将其转换为小写或删除标点符号。
* 您将文本拆分为多个单元（称为“标记”），例如字符、单词或单词组。 这称为标记化。
* 您将每个这样的标记转换为数字向量。 这通常涉及首先索引数据中存在的所有标记。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gt1np624r0j310a0s2jte.jpg)

让我们回顾一下每个步骤。

### 文本标准化

考虑这两个句子：
* “日落来了。我盯着墨西哥的天空。大自然是不是很壮观？？”
* “日落来了，我凝视着墨西哥的天空。大自然是不是很精彩？”

它们非常相似——事实上，它们几乎完全相同。 然而，如果您将它们转换为字节字符串，它们最终会得到非常不同的表示，因为“i”和“I”是两个不同的字符，“Mexico”和“México”是两个不同的词，“isnt”不是 't “不是”，等等。 机器学习模型不知道“i”和“I”是同一个字母，“é”是带重音的“e”，或者“凝视”和“凝视”是两种形式的 同一个动词。

文本标准化是特征工程的一种基本形式，旨在消除您不希望模型必须处理的编码差异。 它也不是机器学习独有的——如果你正在构建一个搜索引擎，你必须做同样的事情。

最简单和最广泛的标准化方案之一是“转换为小写并删除标点符号”。 我们的两句话会变成：
* “日落来了，我凝视着墨西哥的天空，这不是大自然的壮丽景色”
* “日落来了，我凝视着墨西哥的天空，大自然并不灿烂”

已经很近了。 另一种常见的转换是将特殊字符转换为标准形式，例如将“é”替换为“e”，将“æ”替换为“ae”等。 我们的代币“méxico”将变成“mexico”。

最后，在机器学习上下文中很少使用的更高级的标准化模式是词干：将术语的变体（例如动词的不同共轭词干形式）转换为单个共享表示，例如将“捕获”转换为“捕获” 和“一直在捕捉”变成“[catch]”或“cats”变成“[cat]”。 通过词干提取，“was staring”和“stared”会变成类似于“[stare]”的东西——我们两个相似的句子最终会以相同的编码结束：

* “日落来了我[凝视]墨西哥的天空不是大自然的壮丽”

使用这些标准化技术，您的模型将需要更少的训练数据并且可以更好地泛化——它不需要“日落”和“日落”的大量示例来了解它们的含义相同，并且它能够有意义 “México”，即使它在训练集中只看到“mexico”。 当然，标准化也可能会擦除一些信息，因此请始终牢记上下文：例如，如果您正在编写一个从采访文章中提取问题的模型，它肯定应该处理“？” 作为一个单独的令牌而不是丢弃它，因为它是这个特定任务的有用信号。

### 文本分割（标记化）

文本标准化后，您需要将其分解为要向量化的单元（标记），这一步骤称为“标记化”。 您可以通过三种不同的方式执行此操作：
* 单词级标记化，其中标记是空格分隔（或标点分隔）的子字符串。 一种变体是在适用时进一步将单词拆分为子词，例如将“凝视”视为“star+ing”或“call”视为“call+ed”。
* N-gram 标记化，其中标记是 N 个连续单词的组。 例如，“the cat”或“he was”将是 2-gram 标记（也称为 bigrams）。
* 字符级标记化，其中每个字符都是它自己的标记。 在实践中，这种方案很少使用，你只能在专门的上下文中看到它，比如文本生成或语音识别。

通常，您将始终使用词级或 N-gram 标记化。 有两种文本处理模型：一种关心词序，称为“序列模型”，另一种将输入词视为一个集合，丢弃其原始顺序，称为“词袋模型”。 如果您正在构建一个序列模型，您将使用词级标记化，如果您正在构建一个词袋模型，您将使用 N-gram 标记化：N-grams 是一种人工注入的方法 少量的局部词序信息进入模型。 在本章中，您将详细了解每种类型的模型以及何时使用它们。

> **理解 N-gram 和词袋**
>
> 单词 N-gram 是可以从句子中提取的 N 个（或更少）连续单词的组。 相同的概念也可以应用于字符而不是单词。
> 
> 这是一个简单的例子。 考虑句子“the cat sat on the mat”。 它可以分解为以下一组 2-gram：
> ```
> {"the", "the cat", "cat", "cat sat", "sat",
>  "sat on", "on", "on the", "the mat", "mat"}
> ```
> 
> 它也可以分解为以下一组 3-gram：
> ```
> {"the", "the cat", "cat", "cat sat", "the cat sat",
>  "sat", "sat on", "on", "cat sat on", "on the",
>  "sat on the", "the mat", "mat", "on the mat"}
> ```
> 
> 这样的集合分别称为 bag-of-2-grams bag-of-3-grams 或 。这里的术语包指的是您正在处理一组标记而不是列表或序列：标记没有特定的顺序。这一系列标记化方法称为词袋袋N-grams（或）。
> 
> 因为bag-of-words不是保序的tokenization方法（生成的token被理解为一个集合，而不是一个序列，丢失了句子的一般结构），> 所以它倾向于用于浅层语言处理模型而不是深度学习模型。提取 N-gram 是特征工程的一种形式，深度学习序列模型摒弃了这种手动方法，取而代之的是分层特征学习。通过查看连续的单词或字符序列，一维 convnets、循环神经网络和 Transformer 能够学习单词和字符组的表示，而无需明确告知此类组的存在。

### 词汇索引

将文本拆分为标记后，您需要将每个标记编码为数字表示。 您可能会以无状态的方式执行此操作，例如，通过将每个标记散列到一个固定的二进制向量中——但实际上，您要做的方法是构建训练数据中找到的所有术语的索引（ “词汇表”），并为词汇表中的每个条目分配一个唯一的整数。

像这样的东西：

In [ ]:
vocabulary = {}
for text in dataset:
    text = standardize(text)
    tokens = tokenize(text)
    for token in tokens:
        if token not in vocabulary:
            vocabulary[token] = len(vocabulary)

然后，您可以将该整数转换为可由神经网络处理的向量编码，例如单热向量：

In [4]:
def one_hot_encode_token(token):
    vector = np.zeros((len(vocabulary),))
    token_index = vocabulary[token]
    vector[token_index] = 1
    return vector

time: 746 µs (started: 2021-08-01 22:55:45 +08:00)


请注意，在此步骤中，通常会将词汇表限制为仅在训练数据中找到的前 20,000 个或 30,000 个最常见的单词。 任何文本数据集都倾向于包含大量独特的术语，其中大部分只出现一两次——索引这些稀有术语会导致特征空间过大，其中大多数特征几乎没有信息内容。

还记得你在第 4 章和第 5 章中在 IMDB 数据集上训练你的第一个深度学习模型吗？ 您使用的数据来自 keras.datasets.imdb，已经被预处理成整数序列，其中每个整数代表一个给定的单词。 当时，我们使用设置 num_words=10000，以将我们的词汇量限制在训练数据中最常见的前 10,000 个单词。

现在，这里有一个我们不应忽视的重要细节：当我们在词汇索引中查找新标记时，它可能不一定存在。 您的训练数据可能不包含“cherimoya”一词的任何实例（或者您可能因为它太罕见而将其从索引中排除），因此执行 token_index =vocabulary["cherimoya"] 可能会导致 KeyError。 为了解决这个问题，您应该使用“词汇外”索引（缩写为 OOV 索引），它是所有不在索引中的标记的统称。 它通常是索引 1：你实际上是在做 token_index =vocabulary.get(token, 1)。 将整数序列解码回单词时，您将 1 替换为“[UNK]”（您将其称为“OOV 标记”）之类的内容。

“为什么使用 1 而不是 0？”，您可能会问。 那是因为 0 已经被占用了。 您通常会使用两种特殊标记：OOV 标记（索引 1）和“掩码标记”（索引 0）。 虽然 OOV 令牌的意思是“这是一个我们不认识的词”，但掩码令牌告诉我们“忽略我，我不是一个词”。 你会特别用它来填充序列数据：因为数据批次需要是连续的，一批序列数据中的所有序列必须具有相同的长度，所以较短的序列应该填充到最长序列的长度。 如果你想用序列 [5, 7, 124, 4, 89] 和 [8, 34, 21] 制作一批数据，它必须是这样的：
```
[[5, 7, 124, 4, 89]
 [8, 34, 21, 0, 0]]
```

您在第 4 章和第 5 章中使用过的 IMDB 数据集整数序列批次以这种方式填充了零。

### 使用 `TextVectorization` 层

到目前为止，我们介绍的每一步都可以很容易地在纯 Python 中实现。 也许你可以这样写：

In [5]:
import string

class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)
    
    def tokenize(self, text):
        text = self.standardize(text)
        return text.split()
    
    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict((v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]
    
    def decode(self, int_sequence):
        return " ".join(self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)
    
vectorizer = Vectorizer()

dataset = ["I write, erase, rewrite",
           "Erase again, and then",
           "A poppy blooms.",
          ]

vectorizer.make_vocabulary(dataset)

time: 1.66 ms (started: 2021-08-01 23:15:27 +08:00)


它可以完成以下工作：

In [6]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
encoded_sentence

[2, 3, 5, 7, 1, 5, 6]

time: 13.2 ms (started: 2021-08-01 23:16:11 +08:00)


In [7]:
decoded_sentence = vectorizer.decode(encoded_sentence)
decoded_sentence

'i write rewrite and [UNK] rewrite again'

time: 4.49 ms (started: 2021-08-01 23:18:50 +08:00)


然而，使用这样的东西不会有很好的性能。 在实践中，您将使用 Keras TextVectorization 层，它快速高效，可以直接放入 tf.data 管道或 Keras 模型中。

这是 TextVectorization 层的样子：

In [6]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# 配置层以返回编码为整数索引的单词序列。 还有其他几种可用的输出模式，我们将在稍后看到。
text_vectorization = TextVectorization( output_mode="int", )

time: 21.8 ms (started: 2021-08-03 08:18:56 +08:00)


默认情况下，TextVectorization 层将使用设置“转换为小写并删除标点符号”进行文本标准化，并使用“拆分空白”进行标记化。 但重要的是，您可以为标准化和标记化提供自定义函数，这意味着该层足够灵活以处理任何用例。 请注意，此类自定义函数应该对张量进行操作，而不是常规 Python 字符串！ 例如，默认层 tf.string 行为等效于以下内容：

In [6]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
#     将字符串转换为小写
    lowercase_string = tf.strings.lower(string_tensor) 
#     用空字符串替换标点符号
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(string_tensor):
#     在空白处拆分字符串
    return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(output_mode="int",
                                       standardize=custom_standardization_fn,
                                       split=custom_split_fn,)

time: 18.6 ms (started: 2021-08-01 23:31:05 +08:00)


要索引文本语料库的词汇表，只需使用生成字符串的 Dataset 对象或仅使用 Python 字符串列表调用图层的 adapt() 方法：

In [7]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",]

text_vectorization.adapt(dataset)

time: 1.17 s (started: 2021-08-01 23:33:29 +08:00)


请注意，您可以通过 get_vocabulary() 检索计算出的词汇表——如果您需要将编码为整数序列的文本转换回单词，这会很有用。 词汇表中的前两个条目是掩码标记（索引 0）和 OOV 标记（索引 1）。 词汇表中的条目按频率排序——因此对于真实世界的数据集，诸如“the”或“a”之类的非常常见的词将排在第一位。

> 清单 11.1 显示词汇表

In [12]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

time: 22.5 ms (started: 2021-08-01 23:35:14 +08:00)


为了演示，让我们尝试编码然后解码一个例句：

In [14]:
vocabulary = text_vectorization.get_vocabulary()

time: 2.65 ms (started: 2021-08-01 23:36:19 +08:00)


In [15]:
test_sentence = "I write, rewrite, and still rewrite again"

time: 734 µs (started: 2021-08-01 23:36:28 +08:00)


In [17]:
encoded_sentence = text_vectorization(test_sentence)
encoded_sentence

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 7,  3,  5,  9,  1,  5, 10])>

time: 59.3 ms (started: 2021-08-01 23:36:58 +08:00)


In [19]:
inverse_vocab = dict(enumerate(vocabulary))
inverse_vocab

{0: '',
 1: '[UNK]',
 2: 'erase',
 3: 'write',
 4: 'then',
 5: 'rewrite',
 6: 'poppy',
 7: 'i',
 8: 'blooms',
 9: 'and',
 10: 'again',
 11: 'a'}

time: 6.61 ms (started: 2021-08-01 23:37:26 +08:00)


In [20]:
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
decoded_sentence  

'i write rewrite and [UNK] rewrite again'

time: 52.8 ms (started: 2021-08-01 23:37:53 +08:00)


在 tf.data 管道中或作为模型的一部分使用 TextVectorization 层重要的是，因为 TextVectorization 主要是一个字典查找操作，它不能在 GPU（或 TPU）上执行——只能在 CPU 上执行。 因此，如果您在 GPU 上训练模型，那么 TextVectorization 层将在 CPU 上运行，然后将其输出发送到 GPU。 这具有重要的性能影响。

我们可以通过两种方式使用我们的 TextVectorization 层。 第一个选项是将其放入管道中，如下所示：tf.data

> 清单 11.2 选项 1：tf.data 管道中的 TextVectorization

In [ ]:
int_sequence_dataset = string_dataset.map(text_vectorization)

第二种选择是让它成为模型的一部分（毕竟它是一个 Keras 层），就像这样：

> 清单 11.3 选项 2：模型中的 TextVectorization

In [ ]:
# 创建一个需要字符串的符号输入。
text_input = keras.Input(shape=(), dtype="string") 

# 对其应用文本矢量化层。
vectorized_text = text_vectorization(text_input) 

# 您可以继续在顶部链接新层——只是您的常规功能 API 模型。
embedded_input = keras.layers.Embedding(...)(vectorized_text) 
output = ... 

model = keras.Model(text_input, output)

> 两者之间有一个重要区别：如果矢量化步骤是模型的一部分，它将与模型的其余部分同步发生。 这意味着在每个训练步骤中，模型的其余部分（放置在 GPU 上）必须等待 TextVectorization 层（放置在 CPU 上）的输出准备就绪才能开始工作（见图 TODO） . 同时，将该层放入 tf.data 管道中，您可以在 CPU 上对数据进行异步预处理：当 GPU 在一批矢量化数据上运行模型时，CPU 通过矢量化下一批原始字符串来保持忙碌。
> 
> 因此，如果您在 GPU 或 TPU 上训练模型，您可能希望使用选项 1 以获得最佳性能。 这就是我们将在本章中的所有实际示例中执行的操作。 但是，在 CPU 上进行训练时，同步处理很好：无论您选择选项 1 还是选项 2，您都将获得 100% 的内核利用率。
>
> 现在，如果您要将我们的模型导出到生产环境，您可能希望发布一个接受原始字符串作为输入的模型，就像上面选项二的代码片段一样——否则，您将不得不重新实现文本标准化和标记化 您的生产环境（可能在 JavaScript 中？），并且您将面临引入小的预处理差异的风险，这会损害模型的准确性。 值得庆幸的是，TextVectorization 层使您能够将文本预处理直接包含到您的模型中，从而使其更易于部署——即使您最初将该层用作管道的一部分。 在 TODO 部分，您将学习如何将 tf.data 导出为包含文本预处理的仅推理训练模型。

到目前为止，您已经了解了有关文本预处理的所有知识——让我们进入建模阶段。

## 表示词组的两种方法：集合和序列

机器学习模型应该如何表示单个单词是一个相对没有争议的问题：它们是分类特征（来自预定义集合的值），我们知道如何处理这些特征。 它们应该被编码为特征空间中的维度，或作为类别向量（在这种情况下是词向量）。 然而，一个更成问题的问题是如何对单词编织成句子的方式进行编码：词序。

自然语言中的顺序问题是一个有趣的问题：与时间序列的步骤不同，句子中的单词没有自然的规范顺序。 不同的语言以非常不同的方式对相似的单词进行排序。 例如，英语的句子结构与日语的句子结构大不相同。 即使在给定的语言中，您通常也可以通过稍微重新排列单词以不同的方式说出相同的内容。 更进一步，如果你将一个短句中的单词完全随机化，你仍然可以在很大程度上弄清楚它在说什么——尽管在许多情况下似乎会出现明显的歧义。 秩序显然很重要，但它与意义的关系并不简单。

如何表示词序是不同类型 NLP 架构产生的关键问题。 您可以做的最简单的事情就是丢弃顺序并将文本视为一组无序的单词——这为您提供了词袋模型。 您还可以决定严格按照单词出现的顺序处理单词，一次一个，就像时间序列中的步骤一样——然后您可以利用上一章中的循环模型。 最后，混合方法也是可能的：Transformer 架构在技术上与顺序无关，但它将词位置信息注入它处理的表示中，这使其能够同时查看句子的不同部分（与 RNN 不同），同时仍然 订单意识。 因为它们考虑了词序，所以 RNN 和 Transformer 都被称为序列模型。

从历史上看，机器学习在 NLP 中的大多数早期应用只涉及词袋模型。 随着循环神经网络的重生，对序列模型的兴趣在 2015 年才开始上升。 今天，这两种方法仍然适用。 让我们看看它们是如何工作的，以及何时利用它们。

我们将在一个著名的文本分类基准上演示每种方法：IMDB 电影评论情感分类数据集。 在第 4 章和第 5 章中，您已经使用了 IMDB 数据集的预矢量化版本——现在，让我们处理原始 IMDB 文本数据，就像您在现实世界中处理新的文本分类问题时所做的那样。

### 准备 IMDB 电影评论数据

让我们从 Andrew Maas 的斯坦福页面下载数据集开始，并解压缩它：

In [21]:
%%bash

cd data

curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz && tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0   315k      0  0:04:20  0:04:20 --:--:--  321k


time: 6min 31s (started: 2021-08-01 23:53:57 +08:00)


剩下一个名为 aclImdb 的目录，其结构如下：
```
aclImdb/
...train/
......pos/
......neg/
...test/
......pos/
......neg/
```

In [23]:
%%bash

cd data/aclImdb/

time: 99.9 ms (started: 2021-08-02 00:06:29 +08:00)


例如，train/pos/ 目录包含一组 12,500 个文本文件，每个文件都包含用作训练数据的正面情感电影评论的文本正文。 负面情绪评论存在于neg目录中。 总共有 25,000 个文本文件用于负训练，另外 25,000 个用于测试。

那里还有一个 train/unsup 子目录，我们不需要它。 让我们删除它：

In [26]:
![ -d data/aclImdb/train/unsup ] && rm -r data/aclImdb/train/unsup

time: 236 ms (started: 2021-08-02 00:07:50 +08:00)


看看其中一些文本文件的内容。 无论您是处理文本数据还是图像数据，请记住在深入建模之前始终检查数据的外观。 它将为您的模型实际做什么奠定您的直觉。

In [28]:
!cat data/aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoytime: 184 ms (started: 2021-08-02 00:08:32 +08:00)


接下来，让我们通过在新目录 aclImdb/val 中设置 20% 的训练文本文件来准备验证集：

In [107]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("data/aclImdb")
val_dir = base_dir/"val"
train_dir = base_dir/"train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category, exist_ok=True)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files)) 
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir/category/fname, val_dir/category/fname)

time: 3.24 s (started: 2021-08-02 00:58:44 +08:00)


还记得在第 8 章中，我们如何使用实用程序 image_dataset_from_directory 为目录结构创建一批图像及其标签吗？ 您可以使用实用程序 text_dataset_from_directory 对文本文件执行完全相同的数据集操作。 让我们创建三个 Dataset 对象，用于训练、验证和测试：

In [4]:
from tensorflow import keras

batch_size = 32

train_ds = keras.preprocessing.text_dataset_from_directory( 
    "data/aclImdb/train", batch_size=batch_size)

val_ds = keras.preprocessing.text_dataset_from_directory(
    "data/aclImdb/val", batch_size=batch_size)

test_ds = keras.preprocessing.text_dataset_from_directory(
    "data/aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
time: 9.7 s (started: 2021-08-03 08:18:18 +08:00)


运行这一行应该输出“Found 20000 files分为2个类”； 如果您看到“发现 70000 个文件属于 3 个类”，则表示您忘记删除 aclImdb/train/unsup 目录。

这些数据集产生的输入是 TensorFlow 张量，目标是编码值“0”或“1”的 tf.string int32 张量：

> 清单 11.4 显示第一批的形状和数据类型

In [5]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'There\'s a legion of Mick Garris haters out there who feel he couldn\'t direct a horror film of quality if he had to. And, SLEEPWALKERS(..screenplay written by Stephen King)is often used as an example of this. I like SLEEPWALKERS, though I fully am aware that Garris just says F#ck it and lets all hell break loose about fifteen or so minutes into the movie. Forget character or plot development, who needs them anyway. It\'s about violent mayhem and bloody carnage as a mother and son pair of "sleepwalkers"(..feline-human shapeshifting creatures who suck the lifeforce from virginal female innocents, moving from town to town, living a nomadic existence, truly powerful)set their sights on a teenager who doesn\'t surrender without a fight. Before all is said and done, many will be slaughtered as a mother shan\'t tolerate the possible death of her beloved son.<br /><b

可以了，好了。 现在让我们尝试从这些数据中学习一些东西。

### 将单词作为集合处理：词袋方法

对一段文本进行编码以供机器学习模型处理的最简单方法是丢弃顺序并将其视为一组（“袋子”）标记。 您可以查看单个单词（unigrams），也可以通过查看连续标记组（N-grams）来尝试恢复一些本地顺序信息。

**二进制编码的单字（UNIGRAMS）**

如果用一袋单字，句子“the cat sat on the mat”就变成了：
```Python
{"cat", "mat", "on", "sat", "the"}
```

这种编码的主要优点是您可以将整个文本表示为单个向量，其中每个条目都是给定单词的存在指示符。 例如，使用二进制编码（multi-hot），您可以将文本编码为一个向量，其维度与词汇表中的单词数量一样多——0 几乎无处不在，一些 1 表示对文本中存在的单词进行编码的维度 . 这就是我们在第 4 章和第 5 章中处理文本数据时所做的。

让我们在我们的任务中试试这个。

首先，让我们用 TextVectorization 层准备处理我们的原始文本数据集，以便它们产生二进制词向量。 我们的层只会查看单个单词（也就是说，unigrams）。

> 清单 11.5 使用 TextVectorization 层预处理我们的数据集

In [8]:
# 将词汇量限制为 20,000 个最常用的单词。 否则，我们将索引训练数据中的每个单词——可能有数万个只出现一次或两次的术语，
# 因此没有信息量。 一般来说，20,000 是适合文本分类的词汇量。
text_vectorization = TextVectorization(max_tokens=20000, 
                                       output_mode="binary", # 将输出标记编码为二进制向量。
                                      )

# 准备一个仅产生原始文本输入（无标签）的数据集。
text_only_train_ds = train_ds.map(lambda x, y: x) 
# 通过adapter() 方法，使用该数据集来索引数据集词汇表。
text_vectorization.adapt(text_only_train_ds) 

# 准备我们的训练、验证和测试数据集的处理版本。
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y)) 
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

time: 9.48 s (started: 2021-08-03 08:33:31 +08:00)


In [21]:
tf.data.experimental.SqlDataset?

time: 6.27 ms (started: 2021-08-03 09:05:55 +08:00)


Init signature:
tf.data.experimental.SqlDataset(
    driver_name,
    data_source_name,
    query,
    output_types,
)
Docstring:     
A `Dataset` consisting of the results from a SQL query.

`SqlDataset` allows a user to read data from the result set of a SQL query.
For example:

```python
dataset = tf.data.experimental.SqlDataset("sqlite", "/foo/bar.sqlite3",
                                          "SELECT name, age FROM people",
                                          (tf.string, tf.int32))
# Prints the rows of the result set of the above query.
for element in dataset:
  print(element)
```
Init docstring:
Creates a `SqlDataset`.

Args:
  driver_name: A 0-D `tf.string` tensor containing the database type.
    Currently, the only supported value is 'sqlite'.
  data_source_name: A 0-D `tf.string` tensor containing a connection string
    to connect to the database.
  query: A 0-D `tf.string` tensor containing the SQL query to execute.
  output_types: A tuple of `tf.DType` objects r